## Traversing a directory for data

In [2]:
import tensorflow as tf

filenames = tf.train.match_filenames_once('/usr/local/lib/python2.7/dist-packages/bregman/audio/*.wav')
count_num_files = tf.size(filenames)
filename_queue = tf.train.string_input_producer(filenames)
reader = tf.WholeFileReader()
filename, file_contents = reader.read(filename_queue)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_files = sess.run(count_num_files)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    for i in range(num_files):
        audio_file = sess.run(filename)
        print(audio_file)
    coord.request_stop()
    coord.join(threads)

/usr/local/lib/python2.7/dist-packages/bregman/audio/ladokhope.wav
/usr/local/lib/python2.7/dist-packages/bregman/audio/amen.wav
/usr/local/lib/python2.7/dist-packages/bregman/audio/fastcar.wav
/usr/local/lib/python2.7/dist-packages/bregman/audio/gmin.wav
/usr/local/lib/python2.7/dist-packages/bregman/audio/bigben.wav


## Representing audio in Python

In [6]:
from bregman.suite import *

def get_chromogram(audio_file):
    F = Chromagram(audio_file, nfft=16384, wfft=8192, nhop=2205)
    return F.X

## Obtaining a dataset for k-means

In [7]:
chromo = tf.placeholder(tf.float32)
max_freqs = tf.argmax(chromo, 0)

def get_next_chromogram(sess):
    audio_file = sess.run(filename)
    F = Chromagram(audio_file, nfft=16384, wfft=8192, nhop=2205)
    return F.X

def extract_feature_vector(sess, chromo_data):
    num_features, num_samples = np.shape(chromo_data)
    freq_vals = sess.run(max_freqs, feed_dict={chromo: chromo_data})
    hist, bins = np.histogram(freq_vals, bins=range(num_features + 1))
    return hist.astype(float) /num_samples

def get_dataset(sess):
    num_files = sess.run(count_num_files)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    xs = []
    for i in range(num_files):
        chromo_data = get_next_chromogram(sess)
        x = [extract_feature_vector(sess, chromo_data)]
        x = np.matrix(x)
        if len(xs) == 0:
            xs = x
        else:
            xs = np.vstack((xs, x))
    return xs